In [63]:
# Dependencies
!pip install oracledb
!pip install langchain_huggingface
!pip install langchain_community
!pip install langchain  tiktoken
!pip install transformers accelerate bitsandbytes sentence-transformers

In [64]:
# Imports
import oracledb
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings

print("successfully imported libraries")

successfully imported libraries


In [65]:
# Connect to Vector Store
# from google.colab import files
# files.upload()   # choose Wallet_IH9O4WSS1Y1QW6CX.zip

# !unzip Wallet_IH9O4WSS1Y1QW6CX.zip -d wallet
# !pip install oracledb --upgrade

# Connection parameters
username = userdata.get('ORACLE_USERNAME')
password = userdata.get('ORACLE_PASSWORD')
walletPassword = userdata.get('ORACLE_WALLET_PASSWORD')
dsn = "ih9o4wss1y1qw6cx_high"  #DNS from wallet/tnsnames.ora

try:
    conn  = oracledb.connect(
        user=username,
        password=password,
        dsn=dsn,
        config_dir="/content/wallet",  # Wallet folder
        wallet_location="/content/wallet",
        wallet_password=walletPassword
    )
    print("Connection successful!")
except oracledb.Error as e:
    print(f"Could not connect to the database - Error occurred: {str(e)}")

Connection successful!


In [66]:
# Initialize the embedding model
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Test the embedding function
# test_query = "What solutions does the company provide for AI-based document processing?"
# embedding = embed_model.embed_query(test_query)

# Connect to Oracle
knowledge_base = OracleVS(
    client=conn,  # Oracle database connection
    table_name="Company_Solutions",  # Table name where embeddings are stored
    embedding_function=embed_model,  # Embedding function for queries
    distance_strategy=DistanceStrategy.DOT_PRODUCT  # Similarity strategy
)


In [67]:
# Initialize the Language Model (LLM)
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
local_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu


In [68]:
# Retrieval QA
retrieval_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    retriever=knowledge_base.as_retriever(),
    return_source_documents=True
)

In [69]:
# Example user query
user_query = "What details we show on price summary section in home move flow?"

# Retrieve and answer
result = retrieval_chain({"query": user_query})

# Extract the answer and source documents
answer = result["result"]
source_documents = result["source_documents"]

# Print the results
print("Answer:", answer)
print("Source Documents:")
# for doc in source_documents:
#     print(f"Page Content: {doc.page_content}")
#     print(f"Metadata: {doc.metadata}")

Token indices sequence length is longer than the specified maximum sequence length for this model (788 > 512). Running this sequence through the model will result in indexing errors


Answer: Monthly rental - changePlan request totalMonthlyCharge installation charge - homeMove request charges installationCharge Exit charges - (as per existing implmentation) changePlan request mustKnow contractCharges total on first bill - monthly rental + installation charge Early exit charges - same as existing - only static text change which will be prerequisite pay now - as per existing from changePlan request Channel changes (home move with change plan) Screen Requirement / description Journey starts with Homemove BAU Customer select homeMove and provide consent and documents, validate themself For home move with changePlan case, homeMove isHomemoveChangeplan-- true there will be NO move-in for installation date and appointment slot capturing There will be only Move-out date , once captured its shall be updated in Homemove request same as existing this will be final step thereafter in Homemove details screen, moveOut date to be populated with edit option installation Type sectio